In [1]:
import os
import trimesh
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import pysdf
import tinycudann as tcnn # from their github installation
import time
import random

In [2]:
# For running futher code you have to set up your folders and other constants
DATASET_DIR = 'test_task_meshes'
CKPTS_DIR = 'checkpoints'
NUM_POINTS = 500_000
STD_DEVIATION = 1e-2
DEVICE = torch.device('cuda')
EPOCHS = 20
LR = 1e-3

In [3]:
def get_norm_mesh(obj_path):
    mesh = trimesh.load_mesh(obj_path)
    vertices = mesh.vertices
    min_bound, max_bound = mesh.bounds
    center = (min_bound + max_bound) / 2
    scale = 1 / max(max_bound - min_bound)
    mesh.vertices = (vertices - center) * scale
    return mesh

def get_surface_points(mesh, std_dev=STD_DEVIATION, num_points=NUM_POINTS):
    points = np.array(mesh.sample(num_points))
    points += np.random.normal(0, std_dev, points.shape)
    return points

def get_volume_points(num_points=NUM_POINTS):
    points = np.random.rand(num_points, 3) * 2 - 1
    return points

    

def calculate_occupancy_f1(mesh, model, points):
    points = torch.tensor(points, dtype=torch.float32).to(DEVICE)
    sdf_fn = pysdf.SDF(mesh.vertices, mesh.faces)
    gt_sdfs = -sdf_fn(points.cpu().numpy()).reshape(-1, 1).astype(np.float32)
    gt = torch.tensor(gt_sdfs, dtype=torch.float32).to(DEVICE)
    gt = (gt <= 0).to(torch.float32)
    model.eval()
    with torch.no_grad():
        pred_sdfs = model(points)
    pred = (pred_sdfs <= 0).to(torch.float32)
    f1 = (calculate_f1(gt, pred) + calculate_f1(1 - gt, 1 - pred)) / 2
    return f1

def calculate_f1(gt, pred):
    TP = torch.sum((gt == 1) & (pred == 1)).item()
    FN = torch.sum((gt == 1) & (pred == 0)).item()
    FP = torch.sum((gt == 0) & (pred == 1)).item()
    recall = TP + FN
    precision = TP + FP
    if recall:
        recall = TP / recall
    if precision:
        precision = TP / precision
    f1 = recall + precision
    if f1:
        f1 = 2 * (precision * recall) / f1
    return f1




In [4]:
class SDFNetwork(nn.Module):
    def __init__(self,
                 encoding="hashgrid",
                 num_layers=3,
                 skips=[],
                 hidden_dim=64,
                 clip_sdf=None,
                 ):
        super().__init__()

        self.num_layers = num_layers
        self.skips = skips
        self.hidden_dim = hidden_dim
        self.clip_sdf = clip_sdf
        
        assert self.skips == [], 'TCNN does not support concatenating inside, please use skips=[].'

        self.encoder = tcnn.Encoding(
            n_input_dims=3,
            encoding_config={
                "otype": "HashGrid",
                "n_levels": 16,
                "n_features_per_level": 2,
                "log2_hashmap_size": 19,
                "base_resolution": 16,
                "per_level_scale": 1.3819,
            },
        )

        self.backbone = tcnn.Network(
            n_input_dims=32,
            n_output_dims=1,
            network_config={
                "otype": "FullyFusedMLP",
                "activation": "ReLU",
                "output_activation": "None",
                "n_neurons": hidden_dim,
                "n_hidden_layers": num_layers - 1,
            },
        )


    def forward(self, x):
        # x: [B, 3]
        x = (x + 1) / 2 # to [0, 1]
        x = self.encoder(x)
        h = self.backbone(x)
        if self.clip_sdf is not None:
            h = h.clamp(-self.clip_sdf, self.clip_sdf)
        return h
    
def mape_loss(pred, target):
    # pred, target: [B, 1], torch tenspr
    difference = (pred - target).abs()
    scale = 1 / (target.abs() + 1e-2)
    loss = difference * scale

    return loss.mean()

def train_model(mesh, surf_pts, vol_pts, num_epochs=EPOCHS, lr=LR):
    model = SDFNetwork().to(DEVICE)
    criterion = mape_loss
    points_for_train = np.concatenate((surf_pts[:NUM_POINTS//2,:], vol_pts[NUM_POINTS//2:,:]), axis=0)
    
    optimizer = torch.optim.Adam([
        {'name': 'encoding', 'params': model.encoder.parameters()},
        {'name': 'net', 'params': model.backbone.parameters(), 'weight_decay': 1e-6},
    ], lr=lr, betas=(0.9, 0.99), eps=1e-15)
    
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        points = torch.tensor(points_for_train, dtype=torch.float32).to(DEVICE)
        sdf_fn = pysdf.SDF(mesh.vertices, mesh.faces)
        gt_sdf = -sdf_fn(points.cpu().numpy()).reshape(-1, 1).astype(np.float32)
        gt_sdf = torch.tensor(gt_sdf).to(DEVICE)

        pred_sdf = model(points)
        loss = criterion(pred_sdf, gt_sdf)

        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

    return model


In [8]:
surface_scores_f1 = []
volume_scores_f1 = []

for i, obj_name in tqdm(enumerate(os.listdir(DATASET_DIR))):
    pth = os.path.join(DATASET_DIR, obj_name)
    mesh = get_norm_mesh(pth)
    surface_pts = get_surface_points(mesh)
    volume_pts = get_volume_points()
    model = train_model(mesh, surface_pts, volume_pts, EPOCHS, LR)
    surface_points_f1 = calculate_occupancy_f1(mesh, model, surface_pts)
    volume_points_f1 = calculate_occupancy_f1(mesh, model, volume_pts)
    surface_scores_f1.append(surface_points_f1)
    volume_scores_f1.append(volume_points_f1)
    print(f"Boundary points occupancy f1 for {i} sample: {surface_points_f1}")
    print(f"Uniform volume points occupancy f1 for {i} sample: {volume_points_f1}\n")
    
    # uncomment if you want to save separate model
    pth_to_save = os.path.join(CKPTS_DIR, f'model_{i}.pth.tar')
    torch.save({
    'state_dict': model.state_dict()
    }, pth_to_save)
    

0it [00:00, ?it/s]

Epoch 1, Loss: 0.6607466340065002
Epoch 2, Loss: 0.6260046362876892
Epoch 3, Loss: 0.6000893712043762
Epoch 4, Loss: 0.5764374136924744
Epoch 5, Loss: 0.5540878772735596
Epoch 6, Loss: 0.5399360060691833
Epoch 7, Loss: 0.5321506857872009
Epoch 8, Loss: 0.525884747505188
Epoch 9, Loss: 0.5185310244560242
Epoch 10, Loss: 0.5098280906677246
Epoch 11, Loss: 0.5011358857154846
Epoch 12, Loss: 0.4936535954475403
Epoch 13, Loss: 0.48714426159858704
Epoch 14, Loss: 0.48050639033317566
Epoch 15, Loss: 0.47337645292282104
Epoch 16, Loss: 0.46594342589378357
Epoch 17, Loss: 0.4582762122154236
Epoch 18, Loss: 0.45023030042648315
Epoch 19, Loss: 0.44199809432029724
Epoch 20, Loss: 0.4335857927799225
Boundary points occupancy f1 for 0 sample: 0.948198366483372
Uniform volume points occupancy f1 for 0 sample: 0.9642946329763201



1it [00:11, 11.69s/it]

Epoch 1, Loss: 0.663997232913971
Epoch 2, Loss: 0.6437587738037109
Epoch 3, Loss: 0.6163897514343262
Epoch 4, Loss: 0.5925012230873108
Epoch 5, Loss: 0.572019100189209
Epoch 6, Loss: 0.5568759441375732
Epoch 7, Loss: 0.5468201041221619
Epoch 8, Loss: 0.5383155941963196
Epoch 9, Loss: 0.5291266441345215
Epoch 10, Loss: 0.5189071297645569
Epoch 11, Loss: 0.5087898969650269
Epoch 12, Loss: 0.49966493248939514
Epoch 13, Loss: 0.4918428063392639
Epoch 14, Loss: 0.48416993021965027
Epoch 15, Loss: 0.47559118270874023
Epoch 16, Loss: 0.46639537811279297
Epoch 17, Loss: 0.45691055059432983
Epoch 18, Loss: 0.447424978017807
Epoch 19, Loss: 0.4381522536277771
Epoch 20, Loss: 0.42893078923225403
Boundary points occupancy f1 for 1 sample: 0.9540698995651223
Uniform volume points occupancy f1 for 1 sample: 0.9755838256716293



2it [00:18,  8.91s/it]

...

48it [10:58,  7.66s/it]

Epoch 1, Loss: 0.7001197934150696
Epoch 2, Loss: 0.6636274456977844
Epoch 3, Loss: 0.6310111284255981
Epoch 4, Loss: 0.6078565120697021
Epoch 5, Loss: 0.584139883518219
Epoch 6, Loss: 0.5613991022109985
Epoch 7, Loss: 0.549737274646759
Epoch 8, Loss: 0.5443252325057983
Epoch 9, Loss: 0.5385046601295471
Epoch 10, Loss: 0.5310522317886353
Epoch 11, Loss: 0.5218934416770935
Epoch 12, Loss: 0.5121844410896301
Epoch 13, Loss: 0.5034648180007935
Epoch 14, Loss: 0.49642816185951233
Epoch 15, Loss: 0.489986777305603
Epoch 16, Loss: 0.48265159130096436
Epoch 17, Loss: 0.4745985269546509
Epoch 18, Loss: 0.4664662480354309
Epoch 19, Loss: 0.4580119550228119
Epoch 20, Loss: 0.4494364261627197
Boundary points occupancy f1 for 48 sample: 0.9427653356784886
Uniform volume points occupancy f1 for 48 sample: 0.9547050996856574



49it [11:06,  7.71s/it]

Epoch 1, Loss: 0.7632433772087097
Epoch 2, Loss: 0.7134689688682556
Epoch 3, Loss: 0.6782401204109192
Epoch 4, Loss: 0.6571341753005981
Epoch 5, Loss: 0.6344645023345947
Epoch 6, Loss: 0.6046249866485596
Epoch 7, Loss: 0.5778036713600159
Epoch 8, Loss: 0.5641870498657227
Epoch 9, Loss: 0.553246021270752
Epoch 10, Loss: 0.5405564308166504
Epoch 11, Loss: 0.5268561840057373
Epoch 12, Loss: 0.5132336020469666
Epoch 13, Loss: 0.4997406601905823
Epoch 14, Loss: 0.48548880219459534
Epoch 15, Loss: 0.4707809090614319
Epoch 16, Loss: 0.456971675157547
Epoch 17, Loss: 0.4439125657081604
Epoch 18, Loss: 0.430400550365448
Epoch 19, Loss: 0.4173884093761444
Epoch 20, Loss: 0.40607428550720215
Boundary points occupancy f1 for 49 sample: 0.6868718597505963
Uniform volume points occupancy f1 for 49 sample: 0.7727247726997724



50it [11:19, 13.60s/it]


In [9]:
print(f'Average F1 score near the surface points for all dataset: {np.mean(surface_scores_f1)}')

Average F1 score near the surface points for all dataset: 0.9180649626692536


In [10]:
print(f'Average F1 score in the bounding volume points for all dataset: {np.mean(volume_scores_f1)}')

Average F1 score in the bounding volume points for all dataset: 0.9285064997916319


In [12]:
with open('evaluation_f1_scores.txt', 'w') as file:
    for i, obj_name in enumerate(os.listdir(DATASET_DIR)):
        pth_to_obj = os.path.join(DATASET_DIR, obj_name)
        model_name = f'model_{i}.pth.tar'
        pth_to_model = os.path.join(CKPTS_DIR, model_name)
        surface_points_f1 = surface_scores_f1[i]
        volume_points_f1 = volume_scores_f1[i]
        file.write(f"Model: {model_name}, Object: {obj_name}, Surface F1: {round(surface_points_f1, 3)}, Volume F1: {round(volume_points_f1, 3)}\n")
    file.write(f"\nAverage Surface F1: {np.mean(surface_scores_f1)}\n")
    file.write(f"Average Volume F1: {np.mean(volume_scores_f1)}\n")

In [32]:
with open('time_measurement.txt', 'w') as file:
    for model_name in os.listdir(CKPTS_DIR):

        model_pth = os.path.join(CKPTS_DIR, model_name)
        dummy_input = torch.randn(NUM_POINTS, 3, dtype=torch.float).to(DEVICE)

        model = SDFNetwork().to(DEVICE)
        model.load_state_dict(torch.load(model_pth)['state_dict'])


        _ = model(dummy_input) # first initizalize for 'warming up' model

        times = np.zeros((100, 1))
        with torch.no_grad():
            for i in range(100):
                start = time.perf_counter_ns()
                _ = model(dummy_input)
                cur = time.perf_counter_ns() - start
                times[i] = cur

        file.write(f'Model: {model_name} - {np.min(times / 1e6)} milliseconds or {np.min(times)} nanoseconds for {NUM_POINTS} points\n')

C:\Users\Admin\AppData\Local\Temp\ipykernel_10644\3120141458.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_pth)['state_dict'])
